In [1]:
!git clone https://github.com/HarderThenHarder/transformers_tasks
!pip install openai

fatal: destination path 'transformers_tasks' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
%cd transformers_tasks/LLM/llms_mbti

In [6]:
from openai import OpenAI

import pandas as pd
import json
from tqdm.notebook import tqdm

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

In [4]:
with open("mbti_questions.json", "r") as f:
    qa_data = json.load(f)

In [5]:
qa_data

{'1': {'question': '当你要外出一整天时，你会：\\nA.计划要做的事情以及做的时间\\nB.不做计划，说走就走',
  'A': 'J',
  'B': 'P'},
 '2': {'question': '你认为自己是一个什么样的人？\\nA.随性而为的人\\nB.有条不紊的人', 'A': 'P', 'B': 'J'},
 '3': {'question': '假如你是一位老师，你会选教：\\nA.以事实为主的课程\\nB.以理论为主的课程',
  'A': 'S',
  'B': 'N'},
 '4': {'question': '你通常是一个\\nA.善于交际的人\\nB.安静缄默的人', 'A': 'E', 'B': 'I'},
 '5': {'question': '一般来说，你和哪些人比较合得来？\\nA.富于想象力的人\\nB.现实的人',
  'A': 'N',
  'B': 'S'},
 '6': {'question': '你会经常倾向于？\\nA.你的情感支配你的理智\\nB.你的理智主宰你的情感',
  'A': 'F',
  'B': 'T'},
 '7': {'question': '处理许多事情上，你会更喜欢：\\nA.感性地做事\\nB.依逻辑行事', 'A': 'P', 'B': 'J'},
 '8': {'question': '你更倾向于：\\nA.容易让他人了解\\nB.比较难让人了解', 'A': 'E', 'B': 'I'},
 '9': {'question': '严格按照计划做事会让你感到：\\nA.合你心意\\nB.令你感到束缚', 'A': 'J', 'B': 'P'},
 '10': {'question': '当你有一份对你很重要的工作时，你会喜欢：\\nA.在开始前精心组织策划\\nB.在工作进行中找出必要环节',
  'A': 'J',
  'B': 'P'},
 '11': {'question': '在大多数情况下，你会选择：\\nA.顺其自然\\nB.按照计划做事', 'A': 'P', 'B': 'J'},
 '12': {'question': '大多数人认为你是一个\\nA.重视自我隐私的人\\nB.非常坦率开放的人',
  'A': 'I',
  'B': 'E'},
 '

In [42]:
client = OpenAI(
    api_key="PUT YOUR OPENAI API KEY"
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def translate(sentence):
    prompt = f"""Translate the following Chinese sentence into English.
Please convert '\\n' into '\n' and leave '\n' as it is.

{sentence}

Result of translation:
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "You are an excellent translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=100,
        #response_format={"type": "json_object"}, # Output in the format of JSON
    )

    output = response.choices[0].message.content.strip()
    #output_dict = json.loads(response.choices[0].message.content)

    return output

## Translate Chinese QA data into English

In [43]:
results = {}

for id, dp in tqdm(qa_data.items(), total=len(qa_data)):
    result_dict = {}
    result_dict['question'] = translate(dp['question'])
    result_dict['A'] = dp['A']
    result_dict['B'] = dp['B']

    results[id] = result_dict

  0%|          | 0/93 [00:00<?, ?it/s]

In [44]:
original = []
after = []

for i in qa_data:
  original.append(qa_data[i]['question'])
  after.append(results[i]['question'])

In [45]:
df = pd.DataFrame(
    {
        'original': original,
        'after': after
    }
)

df.to_csv('translation_results.csv', index=False)